# 14.6 子词嵌入
- **目录**
  - 14.6.1 fastText模型
  - 14.6.2 字节对编码

- 在英语中，“helps”、“helped”和“helping”等单词都是同一个词“help”的变形形式。
- “dog”和“dogs”之间的关系与“cat”和“cats”之间的关系相同，“boy”和“boyfriend”之间的关系与“girl”和“girlfriend”之间的关系相同。
- 在法语和西班牙语等其他语言中，许多动词有40多种变形形式，而在芬兰语中，名词最多可能有15种变形。
- 在语言学中，形态学研究单词形成和词汇关系。
- 但是，word2vec和GloVe都没有对词的内部结构进行探讨。

## 14.6.1 fastText模型

回想一下词在word2vec中是如何表示的。在跳元模型和连续词袋模型中，同一词的不同变形形式直接由不同的向量表示，不需要共享参数。为了使用形态信息，**fastText模型**提出了一种**子词嵌入**方法，其中子词是一个字符$n$-gram 。fastText可以被认为是子词级跳元模型，而非学习词级向量表示，其中每个**中心词**由其子词级向量之和表示。

让我们来说明如何以单词“where”为例获得fastText中每个中心词的子词。首先，在词的开头和末尾添加特殊字符“&lt;”和“&gt;”，以将前缀和后缀与其他子词区分开来。
然后，从词中提取字符$n$-gram。
例如，值$n=3$时，我们将获得长度为3的所有子词：
“&lt;wh”、“whe”、“her”、“ere”、“re&gt;”和特殊子词“&lt;where&gt;”。

在fastText中，对于任意词$w$，用$\mathcal{G}_w$表示其长度在3和6之间的所有子词与其特殊子词的并集。词表是所有词的子词的集合。假设$\mathbf{z}_g$是词典中的子词$g$的向量，则跳元模型中作为中心词的词$w$的向量$\mathbf{v}_w$是其子词向量的和：

$$\mathbf{v}_w = \sum_{g\in\mathcal{G}_w} \mathbf{z}_g. \tag{14.6.1}$$

fastText的其余部分与跳元模型相同。与跳元模型相比，fastText的词量更大，模型参数也更多。此外，**为了计算一个词的表示，它的所有子词向量都必须求和**，这导致了更高的计算复杂度。然而，**由于具有相似结构的词之间共享来自子词的参数，罕见词甚至词表外的词在fastText中可能获得更好的向量表示**。

- **要点：**
  - **词的表示**：在word2vec中，同一词的不同形态变形是通过不同的向量直接表示的，不共享参数。
  - **子词嵌入**：为了利用形态信息，fastText模型引入了子词嵌入方法。这里的子词指的是字符$n$-gram。
  - **中心词表示**：fastText可以看作是子词级的跳元模型。与其学习词级的向量表示，每个中心词在fastText中是由其子词的向量之和表示的。
  - **子词的提取**：
    - 为词加上特殊字符“<”和“>”标记词的开头和结尾。
    - 提取长度为$n$的字符子词。例如，对于词“where”且$n=3$，我们有子词：“<wh”、“whe”、“her”、“ere”、“re>”和特殊子词“<where>”。
  - **子词集合**：对于任何词$w$，$\mathcal{G}_w$表示其所有长度在3和6之间的子词加上它的特殊子词的集合。
  - **词表**：是所有词的子词的集合。词$w$的向量表示$\mathbf{v}_w$是其所有子词向量的和，如公式\(14.6.1\)所示。
  - **与跳元模型的对比**：
    - fastText的词量更大，有更多的模型参数。
    - 计算一个词的表示需要求所有子词向量的和，导致更高的计算复杂度。
    - 但由于形态相似的词可以共享子词参数，所以罕见词和词表外的词在fastText中可能获得更好的向量表示。

-------------
- **说明：**
- **（1）何为子词，与单词有何区别与联系？**
  - **子词 (Subword)** 和 **单词 (Word)** 是自然语言处理和语言学中常用的概念，它们之间有一些基本的区别和联系：
  - **定义：**
    - **子词 (Subword)：** 它通常是单词的一部分，可以是单词中的连续字符的任意组合。
      - 在某些语言或模型中，子词可以是有意义的，例如在英语中，“unhappiness”包含子词“un-”(表示否定)和“-ness”(表示状态或质量)。
      - 但在其他场合，子词可能并没有明确的意义，例如上面提到的字符n-gram中的“whe”。
    - **单词 (Word)：** 在大多数情况下，它是语言的基本单位，具有特定的意义或功能。例如，“happiness”是一个单词，表示一种情感或状态。
  - **关系：**
    - 一个单词可以分解为多个子词。
    - 子词组合在一起可以构成一个单词。
  - **使用场景：**
    - **子词：** 在处理像德语或芬兰语这样的合成语言时，使用子词可以非常有用，因为这些语言中的单词可能由多个词根组成，具有复杂的结构。另外，在处理大量未知或罕见词汇时，例如在机器翻译中，子词嵌入可以提供更好的表示。
    - **单词：** 在许多自然语言处理任务中，如文本分类、情感分析和命名实体识别等，通常使用单词级别的表示。
  - 优缺点：
    - **子词优点：** 能够捕获词的形态信息，有助于处理未知词和罕见词，以及处理具有丰富形态变化的语言。
    - **单词优点：** 直接和语言的语义和句法结构相关，易于解释。
<br><br>
- **（2）罕见词以及词表外词共享子词参数的例子**
  - 考虑英语中的词“neologism”和“neology”。
    - **“neologism”** 是一个罕见词，意思是新创造的词或短语。
    - **“neology”** 这个词不如“neologism”那么罕见，它指的是新的言辞或表达方式。
  - 假设训练语料库中只包含“neology”，但想获取“neologism”这个词在fastText模型中的表示。
    - 在fastText中，单词被分解为子词。因此，“neology”可能被分解为以下子词（假设我们考虑长度为3和4的字符n-gram）:
    - &lt;neo, neol, eolo, olog, logy, ogy&gt;, <neolo, eolog, ology>
  - 如果我们尝试为“neologism”获取表示，尽管它可能不在我们的训练语料库中，但它可以被分解为以下子词：
    - &lt;neol, eolo, olog, logi, ogis, gism, ism&gt;, <neolo, eolog, ologi, logis, ogism>
    - 可以看到，“neologism”与“neology”有多个公共的子词，例如&lt;neol, eolo, olog等。
      - 这意味着即使“neologism”没有出现在训练数据中，其表示仍然可以通过与“neology”共享的子词向量来构建。
  - 这种子词共享使得fastText能够为罕见词和词表外的词提供合理的向量表示。
-------------

## 14.6.2 字节对编码（Byte Pair Encoding）
- 在fastText中，所有提取的子词都必须是指定的长度，例如$3$到$6$，因此词表大小不能预定义。
- 为了在固定大小的词表中允许可变长度的子词，可以应用一种称为**字节对编码（Byte Pair Encoding，BPE）**的压缩算法来提取子词。
  - 字节对编码执行训练数据集的统计分析，以发现单词内的公共符号，诸如任意长度的连续字符。
  - 从长度为1的符号开始，字节对编码**迭代地合并最频繁的连续符号对**以产生新的更长的符号。
    - 请注意，为提高效率，不考虑跨越单词边界的对。
  - 最后可以使用像子词这样的符号来切分单词。
  - 字节对编码及其变体已经用于诸如GPT-2和RoBERTa等自然语言处理预训练模型中的输入表示。

- 下面将说明字节对编码是如何工作的。
- 首先，此处将符号词表初始化为所有英文小写字符、特殊的词尾符号`'_'`和特殊的未知符号`'[UNK]'`。

In [74]:
import collections

symbols = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
           'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z',
           '_', '[UNK]']

- 因为不考虑跨越词边界的符号对，所以我们只需要一个字典`raw_token_freqs`将词映射到数据集中的频率（出现次数）。
- 注意，特殊符号`'_'`被附加到每个词的尾部，以便可以容易地从输出符号序列
  - （例如，“a_all er_man”）恢复单词序列（例如，“a_all er_man”）。
- 由于我们仅从单个字符和特殊符号的词开始合并处理，所以在每个词（词典`token_freqs`的键）内的每对连续字符之间插入空格。
- 换句话说，空格是词中符号之间的分隔符。


In [75]:
raw_token_freqs = {'fast_': 4, 'faster_': 3, 'tall_': 5, 'taller_': 4}
token_freqs = {}
for token, freq in raw_token_freqs.items():
    token_freqs[' '.join(list(token))] = raw_token_freqs[token]
token_freqs

{'f a s t _': 4, 'f a s t e r _': 3, 't a l l _': 5, 't a l l e r _': 4}

- 定义以下`get_max_freq_pair`函数：
  - 该函数返回词内最频繁的连续符号对。
  - 其中词来自输入词典`token_freqs`的键。


In [76]:
## 原代码
def get_max_freq_pair(token_freqs):
    pairs = collections.defaultdict(int)
    for token, freq in token_freqs.items():
        symbols = token.split()
        for i in range(len(symbols) - 1):
            # “pairs”的键是两个连续符号的元组
            pairs[symbols[i], symbols[i + 1]] += freq
    return max(pairs, key=pairs.get)  # 具有最大值的“pairs”键

In [77]:
## 注释代码
## 接受一个词频词典token_freqs作为参数
def get_max_freq_pair(token_freqs):
    '''
    使用collections.defaultdict初始化一个默认词典pairs。
    该词典用于存储符号对及其出现的频率。
    defaultdict(int)意味着任何尚未存在于词典中的键都会默认具有值int()，也就是0。
    '''
    pairs = collections.defaultdict(int)
    ## 开始遍历token_freqs词典的每个条目，其中token是词，freq是词的出现频率
    for token, freq in token_freqs.items():
        ## 将token（一个由空格分隔的符号字符串）拆分成一个符号列表，并赋值给symbols
        symbols = token.split()
        ## 遍历symbols列表以便查找两个连续符号，但停在倒数第二个符号
        for i in range(len(symbols) - 1):
            # “pairs”的键是两个连续符号的元组
            '''
            更新pairs词典中当前连续符号对的频率。
            键是符号对的元组(symbols[i], symbols[i + 1])，增加该键对应的频率值freq。
            '''
            pairs[symbols[i], symbols[i + 1]] += freq
    #print('paris-------', pairs)
    '''
    在所有连续符号对中返回具有最大频率的那对。
    key=pairs.get意味着使用pairs词典的值（即频率）作为决定“最大”的标准。
    '''
    return max(pairs, key=pairs.get)  # 具有最大值的“pairs”键

- 作为基于连续符号频率的贪心方法，字节对编码将使用以下`merge_symbols`函数来合并最频繁的连续符号对以产生新符号。


In [78]:
## 原代码
def merge_symbols(max_freq_pair, token_freqs, symbols):
    symbols.append(''.join(max_freq_pair))
    new_token_freqs = dict()
    for token, freq in token_freqs.items():
        new_token = token.replace(' '.join(max_freq_pair),
                                  ''.join(max_freq_pair))
        new_token_freqs[new_token] = token_freqs[token]
    return new_token_freqs

In [79]:
## 注释代码
'''
两种解释：
（1）合并最高频率的符号对，并更新词频词典来反映这种合并。
这是字节对编码（BPE）算法中的一个关键步骤，
它持续地合并最常见的符号对，直到达到所需的词汇大小或其他停止条件。
（2）将token_freqs中出现频率最高的符号对max_freq_pair合并，
并更新token_freqs词典来反映这种合并。
同时，新的合并符号被添加到symbols列表中。
'''
## 三个参数：最高频率的符号对max_freq_pair、词频词典token_freqs和符号列表symbols。
def merge_symbols(max_freq_pair, token_freqs, symbols):
    '''
    码将max_freq_pair中的两个符号合并成一个字符串，
    然后将其添加到symbols列表的末尾。
    例如，如果max_freq_pair是('a', 'b')，
    则它会被合并为'ab'并添加到symbols列表中。
    '''
    symbols.append(''.join(max_freq_pair))
    ## 新词典对象用于存储合并后的词及其频率
    new_token_freqs = dict()
    
    for token, freq in token_freqs.items():
        '''
        替换词中的最高频率的符号对。
        它首先将max_freq_pair中的符号转换为由空格分隔的字符串（例如，'a b'），
        然后在token中找到这个字符串并将其替换为无空格的版本（例如，'ab'）。
        结果存储在new_token中。
        '''
        new_token = token.replace(' '.join(max_freq_pair),
                                  ''.join(max_freq_pair))
        ## 将原始频率从token_freqs复制到新词典new_token_freqs，以new_token为键。
        new_token_freqs[new_token] = token_freqs[token]
    return new_token_freqs

- 现在对词典`token_freqs`的键迭代地执行字节对编码算法。
  - 在第一次迭代中，最频繁的连续符号对是`'t'`和`'a'`，因此字节对编码将它们合并以产生新符号`'ta'`。
  - 在第二次迭代中，字节对编码继续合并`'ta'`和`'l'`以产生另一个新符号`'tal'`。


In [80]:
num_merges = 10
for i in range(num_merges):
    max_freq_pair = get_max_freq_pair(token_freqs)
    token_freqs = merge_symbols(max_freq_pair, token_freqs, symbols)
    print(f'合并# {i+1}:',max_freq_pair)

合并# 1: ('t', 'a')
合并# 2: ('ta', 'l')
合并# 3: ('tal', 'l')
合并# 4: ('f', 'a')
合并# 5: ('fa', 's')
合并# 6: ('fas', 't')
合并# 7: ('e', 'r')
合并# 8: ('er', '_')
合并# 9: ('tall', '_')
合并# 10: ('fast', '_')


- 在字节对编码的10次迭代之后，可以看到列表`symbols`现在又包含10个从其他符号迭代合并而来的符号。


In [81]:
print(symbols)

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '_', '[UNK]', 'ta', 'tal', 'tall', 'fa', 'fas', 'fast', 'er', 'er_', 'tall_', 'fast_']


- 对于在词典`raw_token_freqs`的键中指定的同一数据集，作为字节对编码算法的结果，数据集中的每个词现在被子词“fast_”、“fast”、“er_”、“tall_”和“tall”分割。
  - 例如，单词“fast er_”和“tall er_”分别被分割为“fast er_”和“tall er_”。


In [83]:
print(list(token_freqs.keys()))

['fast_', 'fast er_', 'tall_', 'tall er_']


- 请注意，字节对编码的结果取决于正在使用的数据集。
- 还可以使用从一个数据集学习的子词来切分另一个数据集的单词。
- 作为一种贪心方法，下面的`segment_BPE`函数尝试将单词从输入参数`symbols`分成可能最长的子词。


In [8]:
def segment_BPE(tokens, symbols):
    outputs = []
    for token in tokens:
        start, end = 0, len(token)
        cur_output = []
        # 具有符号中可能最长子字的词元段
        while start < len(token) and start < end:
            if token[start: end] in symbols:
                cur_output.append(token[start: end])
                start = end
                end = len(token)
            else:
                end -= 1
        if start < len(token):
            cur_output.append('[UNK]')
        outputs.append(' '.join(cur_output))
    return outputs

- 使用列表`symbols`中的子词（从前面提到的数据集学习）来表示另一个数据集的`tokens`。


In [9]:
tokens = ['tallest_', 'fatter_']
print(segment_BPE(tokens, symbols))

['tall e s t _', 'fa t t er_']


## 小结

* fastText模型提出了一种子词嵌入方法：基于word2vec中的跳元模型，它将中心词表示为其子词向量之和。
* 字节对编码执行训练数据集的统计分析，以发现词内的公共符号。作为一种贪心方法，字节对编码迭代地合并最频繁的连续符号对。
* 子词嵌入可以提高稀有词和词典外词的表示质量。